In [1]:
import numpy as np
import pandas as pd
# Entorno Scikit Learn
from sklearn.neighbors import KNeighborsClassifier   # Algoritmo
from sklearn import metrics      # Metrica de desempeño ( para ver que tan bien se ajusta nuestro modelo a los datos)
from sklearn.model_selection import train_test_split    # Para dividir el conjunto de prueba y entrenamiento
from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score   # Validación cruzada 
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn import preprocessing           # Libreria para reescalar los datos
from sklearn.impute import SimpleImputer    # Libreria para trabajar con datos faltantes 
# Para gráficos
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# datos
data=pd.read_csv("https://raw.githubusercontent.com/Albertuff/Machine-Learning/master/datos/Telco-Customer-Churn.csv")
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
data.shape

(7043, 21)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [5]:
# Totalcharges es de tipo numerica, sin embargo están como tipo objeto. Vamos a convertir la variable a numerica
data.TotalCharges=pd.to_numeric(data.TotalCharges,errors="coerce")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [6]:
# La variable target debe ser de tipo numerica, la transformamos:
data.Churn=(data.Churn=="Yes").astype(int)      # Si la condicion es verdadera el tipo de dato es 1, si es falsa entonces devuelve un cero

In [7]:
pd.value_counts(data.Churn)

0    5174
1    1869
Name: Churn, dtype: int64

In [8]:
# Notamos cierto nivel de desvalanceo en la variable de respuesta. Será importante balancear los datos para evitar "trampas del algoritmo"
# la trampa es que prediga la clase mas frecuente.
# Importante: El balanceo es parte del pre-procesamiento

# Seleccionamos los atributos
X=data[["tenure","MonthlyCharges","TotalCharges"]]
Y=data["Churn"]
nombres=X.columns

In [9]:
# Seleccionamos el mejor modelo a través de una validación cruzada y la busqueda de la rejilla de calibración
# Se empleará: Una técnica para reemplazar los datos faltantes y otra para balancear los datos
from imblearn.pipeline import make_pipeline     # Para la el pipeline
from imblearn.over_sampling import SMOTE        # Implemente la técnica de SMOTE para crear muestras artificiales para balancear los datos

# Vamos a definir el pipe.
pipe=make_pipeline(SimpleImputer(strategy="median"),        # Para reemplazar datos faltantes
                   SMOTE(random_state=1234),                # Para balancear los datos "completos"
                   KNeighborsClassifier()                   # Algoritmo que queremos calibrar
                  )

# Espacio de búsqueda (hiperparámetros)
parametros={"kneighborsclassifier__n_neighbors":np.arange(1,50)}

# Definimos el método de búsqueda del mejor hiperparámetro
rejilla=GridSearchCV(pipe,param_grid=parametros,scoring="recall",cv=10,n_jobs=-1).fit(X,Y)

print(f" La mejor calibración del modelo es: {rejilla.best_params_}")
print(f" La presición mas alta es de: {rejilla.best_score_}")

 La mejor calibración del modelo es: {'kneighborsclassifier__n_neighbors': 37}
 La presición mas alta es de: 0.7228278994882411


El modelo de k=37 tiene una sensibilidad del 72.82%. Esto es, predice Churn=1, en el 72.82% de los clientes que realmente cancelan su servicio

In [10]:
# Procedemos ahora a entrenar el modelo final con todos los datos
# Previo a entrenar el modelo final, tenemos que imputar los datos faltantes y balancear los datos

# 1.- Imputar los datos faltantes
imput_estrategia=SimpleImputer(strategy="median")

# 2.- Aprender la estrategia en el conjunto de datos
imput_estrategia.fit(X)

# 3.- Aplicamos la estrategia en el conjunto de datos
X=imput_estrategia.transform(X)                      # X ya no tiene datos faltantes, pero ya no es un dataframe sino un array
X=pd.DataFrame(X,columns=nombres)

# 4.- Balanceamos la base de datos
smote=SMOTE(sampling_strategy="minority",random_state=1234)
X_smote,Y_smote=smote.fit_resample(X,Y)             # X_smote está balanceada y es un dataframe

# 5.- Se utiliza la muestra balanceada para entrenar el modelo final
modelo_final=KNeighborsClassifier(n_neighbors=37).fit(X_smote,Y_smote)


In [11]:
# Este modelo final, ya entrenado nos sirve para hacer predicciones
# Vamos a crear un dataframe con datos nuevos sobre los cuales deseamos hacer predicciones

datos_nuevos=pd.DataFrame()
datos_nuevos["tenure"]=[6,13,40]
datos_nuevos["MonthlyCharges"]=[60,130,100]
datos_nuevos["TotalCharges"]=[125," ",1300]

# El objetivo es predecir si los clientes abandonan 
# Vamos a hacer el preprocesamiento de datos:

# Hacemos que el tipo de datos sean compatibles
datos_nuevos.TotalCharges=pd.to_numeric(datos_nuevos.TotalCharges,errors="coerce")

# Completamos los datos faltantes empleando la estrategia imput que aprendio en el entrenamiento
datos_nuevos_completos=pd.DataFrame(imput_estrategia.transform(datos_nuevos),columns=nombres)

# Hacemos las predicciones
predicciones=modelo_final.predict(datos_nuevos_completos)
datos_nuevos["Churn"]=predicciones
datos_nuevos

,tenure,MonthlyCharges,TotalCharges,Churn
0,6,60,125.0,1
1,13,130,NaN,1
2,40,100,1300.0,1
